# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева*

## Практикум 1.2. Парсинг с `BeautifulSoup`: тэги и атрибуты

Мы уже немного познакомились со структурой html-страниц и поиском информации по тэгам, теперь попробуем выгрузить информацию из более интересной страницы, а точнее, с сайта [nplus1.ru](https://nplus1.ru/). Наша задача – выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в файл Excel.

Для работы нам снова понадобится модуль `requests` для отправки запросов, для «подключения» к странице и получения ее содержимого в виде строки, и функция `BeautifulSoup` из библиотеки `bs4` для удобного поиска по полученной строке:

In [1]:
import requests
from bs4 import BeautifulSoup

Сохраним ссылку на главную страницу в переменную `main` и отправим запрос к ней с помощью функции `get()` из `requests`:

In [2]:
main = "https://nplus1.ru/"
page = requests.get(main)

Заберём исходный код страницы и преобразуем строку с ним в объект `BeautifulSoup`:

In [3]:
soup = BeautifulSoup(page.text)

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. Найдем кусочки кода HTML, соответствующие всем ссылкам на главной странице сайта:

In [4]:
links_raw = soup.find_all("a")
links_raw[10:20]  # несколько штук для примера

[<a class="hover:underline transition-colors duration-75" href="https://nplus1.ru/search?tags=880">Астрономия</a>,
 <a class="hover:underline transition-colors duration-75" href="https://nplus1.ru/search?tags=768">Генетика</a>,
 <a class="hover:underline transition-colors duration-75" href="https://nplus1.ru/search?tags=890">Математика</a>,
 <a class="hover:underline transition-colors duration-75" href="https://nplus1.ru/search?tags=871">Космонавтика</a>,
 <a class="hover:underline transition-colors duration-75" href="https://nplus1.ru/search?tags=876">Археология</a>,
 <a class="hover:underline transition-colors duration-75" href="https://nplus1.ru/search?tags=775">Нейронауки</a>,
 <a class="hover:underline transition-colors duration-75" href="https://nplus1.ru/search?tags=767">На мышах</a>,
 <a class="hover:underline transition-colors duration-75" href="https://nplus1.ru/search?tags=771">Звук</a>,
 <a class="hover:underline transition-colors duration-75" href="https://nplus1.ru/search

Каждый элемент возвращаемого списка имеет тип `BeautifulSoup` и структуру, очень похожую на словарь. Например, ссылка `<a class="hover:underline transition-colors duration-75" href="/search/empty/768">Генетика</a>` изнутри выглядит как словарь следующего вида:

    {'href' : '/search/empty/768',
     'class' : 'hover:underline transition-colors duration-75'}.
    
Как мы помним, значение по ключу из словаря можно вызвать с помощью метода `.get()`. Давайте извлечем значения по ключу `href` из каждого элемента списка `links`:

In [5]:
links = [li.get("href") for li in links_raw]
links[10:20]  # несколько штук для примера

['https://nplus1.ru/search?tags=880',
 'https://nplus1.ru/search?tags=768',
 'https://nplus1.ru/search?tags=890',
 'https://nplus1.ru/search?tags=871',
 'https://nplus1.ru/search?tags=876',
 'https://nplus1.ru/search?tags=775',
 'https://nplus1.ru/search?tags=767',
 'https://nplus1.ru/search?tags=771',
 'https://nplus1.ru/search?tags=772',
 'https://nplus1.ru/search?tags=778']

Ссылок в списке выше много. Но нам нужны только новости – ссылки, которые начинаются с `https://nplus1.ru/news`. Создадим пустой список `news` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [6]:
news = []
for li in links:
    if "https://nplus1.ru/news/" in li:
        news.append(li)

In [7]:
news[0:10]

['https://nplus1.ru/news/2015/09/21/editor-thy-name',
 'https://nplus1.ru/news/2024/01/19/bempedoic-acid',
 'https://nplus1.ru/news/2024/01/19/peregrine-fall',
 'https://nplus1.ru/news/2024/01/19/iup-in-china',
 'https://nplus1.ru/news/2024/01/19/pig-liver',
 'https://nplus1.ru/news/2024/01/19/ulfberht-sword',
 'https://nplus1.ru/news/2024/01/19/bottom-trawling',
 'https://nplus1.ru/news/2024/01/19/dna-from-birch-tar',
 'https://nplus1.ru/news/2024/01/19/antechinus-mimetes',
 'https://nplus1.ru/news/2024/01/19/just-do-not-smoke']

Первая ссылка ведет не совсем на новость, скорее, на объявление, поэтому давайте ее уберем:

In [8]:
news = news[1:]

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `news` в цикле. Посмотрим на новость с индексом 0, у вас может быть другая, новости обновляются.

In [9]:
link0 = news[0]
print(link0)

https://nplus1.ru/news/2024/01/19/bempedoic-acid


### Задача 1

Отправьте запрос к странице по ссылке `link0` с одной новостью, получите результат в виде объекта `BeautifulSoup` и сохраните его как `soup0`.

In [10]:
page0 = requests.get(link0)
soup0 = BeautifulSoup(page0.text)

### Задача 2

Найдите заголовок новости и сохраните его в переменную `title`.

In [12]:
title = soup0.find("title").text
title = title.replace("\xa0", " ") # заменяем неразрывный пробел на обычный
print(title)

Бемпедоевая кислота снизила риск инфаркта и инсульта. И других серьезных сердечно-сосудистых событий


### Задача 3

Найдите имя автора новости и дату её публикации. Сохраните их в `author` и `date` соответственно.

In [13]:
# находим все тэги meta
soup0.find_all("meta")

[<meta charset="utf-8"/>,
 <meta content="width=device-width, initial-scale=1" name="viewport"/>,
 <meta content="#f26e40" name="msapplication-TileColor"/>,
 <meta content="#ffffff" name="theme-color"/>,
 <meta content="8c90b02c84ac3b72" name="yandex-verification"/>,
 <meta content="b419949322895fc9106e24ed01be58ac" name="pmail-verification"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" name="description"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" property="og:site_name"/>,
 <meta content="Бемпедоевая кислота снизила риск инфаркта и инсульта" property="og:title"/>,
 <meta content="https://minio.nplus1.ru/app-images/879086/65a95dcee874a_cover_share.png" property="og:image"/>,
 <meta content="https://nplus1.ru/news/2024/01/19/bempedoic-acid" property="og:url"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" property="og:description"/>,
 <meta content="article" property="og:type"/>,
 <meta content="2024

In [14]:
# уточняем критерии поиска
# указываем значение атрибута itemprop

soup0.find("meta", {"itemprop" : "datePublished"})

<meta content="2024-01-19" itemprop="datePublished"/>

In [15]:
# забираем содержимое из атрибута content
# по анологии со ссылкой и href

soup0.find("meta", {"itemprop" : "datePublished"}).get("content")

'2024-01-19'

In [16]:
# все вместе и аналогично для автора

date = soup0.find("meta", {"itemprop" : "datePublished"}).get("content")
author = soup0.find("meta", {"name" : "mediator_author"}).get("content")

print(date, author)

2024-01-19 Слава Гоменюк


### Задача 4

Найдите сложность новости и рубрики, к которым она относится. Сохраните сложность в переменную `diffc`. Рубрики сначала можно сохранить в список, а затем его элементы объединить в одну строку `rubs`.

**Подсказка:** чтобы упростить себе жизнь, найдите сначала раздел (`div`), в котором на странице хранится дата и время публикации, сложность и рубрики, а затем выполняйте поиск в рамках этого раздела.

In [ ]:
# находим подходящий фрагмент кода

div = soup0.find("div",
                 {"class" : "flex flex-wrap lg:mb-10 gap-2 text-tags xl:pr-9"})

In [ ]:
# находим внутри этого фрагмента четвертый элемент с тэгом span
# в нем хранится сложность

diffc = div.find_all("span")[3].text

In [ ]:
# при желании можем сразу превратить результат в число типа float, но пока не будем
# сделаем после сбора всех значений

float(div.find_all("span")[3].text)

2.5

In [ ]:
# находим рубрики – все элементы после сложности, может быть несколько
# извлекаем текст рубрик
# склеиваем в одну строку

rubs_raw = div.find_all("span")[4:]
rubs_list = [r.text for r in rubs_raw]
rubs = ", ".join(rubs_list)
rubs

'Медицина, Биология'

### Задача 5

Соберите из абзацев текст новости и сохраните его в переменную `text`. Избавьтесь от постронних символов (`\xa0`, `\n`) в тексте.

In [ ]:
# ищем все абзацы с классом mb-6, чтобы не забрать лишний текст
# извлекаем текст
# склеиваем все абзацы в один большой текст
# заменяем символы через .replace()
# \xa0 – неразрывные пробелы, \n - переходы на новую строку

pars_raw = soup0.find_all("p", {"class" : "mb-6"})
pars = [p.text for p in pars_raw]
text = " ".join(pars)
text = text.replace("\xa0", " ").replace("\n", " ")
text

'Технология помогла мышам лучше восстановиться после ношения гипса Биоинженеры из Гарвардского университета наносили на мышцы гель-эластомер с пружиной внутри и таким образом воспроизводили сокращение и растяжение мышц в обездвиженных конечностях. Технология защитила мышей от атрофии конечностей и запустила рост мышечной ткани. Работа опубликована в журнале Nature Materials. Атрофия мышц — это процесс, при котором происходит истончение мышечных волокон и уменьшение объема мышцы. Атрофия быстро наступает при недостатке физической нагрузки, например, после длительного ношения гипса или у лежачих больных. Это происходит потому, что в обездвиженных мышцах ухудшается кровоток и доставка питательных веществ. А еще мышцам крайне важно постоянно испытывать механическое напряжение, а когда сигналов о сокращении и растяжении мышц нет, организм перестает перестает поддерживать мышечную ткань в рабочем состоянии.  Обездвиженным людям пытаются помочь механотерапией — массажем или компрессионными ус

### Задача 6

Напишите функцию `get_news()`, которая принимает на вход ссылку на страницу с одной новостью, а возвращает список из следующих характеристик: имя автора, дата публикации, сложность новости, рубрики, текст новости.

In [ ]:
def get_news(link0):
    page0 = requests.get(link0)
    soup0 = BeautifulSoup(page0.text)
    title = soup0.find("title").text
    author = soup0.find("meta", {"name" : "mediator_author"}).get("content")
    date = soup0.find("meta", {"itemprop" : "datePublished"}).get("content")
    div = soup0.find("div",
                 {"class" : "flex flex-wrap lg:mb-10 gap-2 text-tags xl:pr-9"})
    diffc = div.find_all("span")[3].text
    rubs_raw = div.find_all("span")[4:]
    rubs_list = [r.text for r in rubs_raw]
    rubs = ", ".join(rubs_list)
    pars_raw = soup0.find_all("p", {"class" : "mb-6"})
    pars = [p.text for p in pars_raw]
    text = " ".join(pars)
    text = text.replace("\xa0", " ").replace("\n", " ")
    return [title, author, date, diffc, rubs, text]

Отлично! Теперь осталось применить функцию ко всем ссылкам в списке `news`. Чтобы сайт не понял, что мы его автоматически грабим, будем выгружать новости постепенно – с задержкой в 1.5 секунды. Импортируем для этого функцию `sleep` :

In [ ]:
from time import sleep

Теперь будем применять функцию в цикле к каждой ссылке в `news`, только с одним дополнением – добавленной конструкцией `try-except`, которая позволит продолжать исполнение цикла, если при применении функции Python столкнулся с ошибкой любого вида:

In [ ]:
info = []
for n in news:
    # пробуй исполнить следующий код
    try:
        res = get_news(n)
        info.append(res)
        print(n)
    # если он вызвал ошибку, печатай сообщение и иди дальше
    except:
        print("Something went wrong")
        print(n)
    sleep(1.5)

https://nplus1.ru/news/2023/01/16/MAGENTA-anti-atrophy
https://nplus1.ru/news/2023/01/17/restoration-of-spermatogenesis
https://nplus1.ru/news/2023/01/17/healthy-mice-reprogrammed
https://nplus1.ru/news/2023/01/17/tursiops-truncatus
https://nplus1.ru/news/2023/01/17/CMH-adolescent
https://nplus1.ru/news/2023/01/17/Molecule-in-SlowMo
https://nplus1.ru/news/2023/01/17/SLYM-meninge
https://nplus1.ru/news/2023/01/17/child-with-dogs
https://nplus1.ru/news/2023/01/17/singing-intervention-aphasia
https://nplus1.ru/news/2023/01/17/ppnb
https://nplus1.ru/news/2023/01/17/anti-Nogo-A
https://nplus1.ru/news/2023/01/16/walliserops-trifurcatus
https://nplus1.ru/news/2023/01/16/Mister-Bubbles
https://nplus1.ru/news/2023/01/16/like-share
https://nplus1.ru/news/2023/01/16/marriage-in-minoan-crete
https://nplus1.ru/news/2023/01/16/found-in-translation
https://nplus1.ru/news/2023/01/16/da5-mice
https://nplus1.ru/news/2023/01/16/royal-tomb
https://nplus1.ru/news/2023/01/14/toi-700-e
https://nplus1.ru/news

Посмотрим на несколько элементов `info`:

In [ ]:
#info[10:12]

Финальный штрих – импортируем `pandas` и преобразуемый полученный список кортежей в датафрейм:

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(info)
df.head()

,0,1,2,3,4,5
0,Сокращающийся имплантат защитил мышцы от атрофии,Амина Садреева,2023-01-16,2.5,"Медицина, Биология",Технология помогла мышам лучше восстановиться ...
1,Транс-женщины восстановили сперматогенез после...,Слава Гоменюк,2023-01-17,2.6,Медицина,Их партнеры смогли забеременеть После отмены з...
2,Репрограммирование клеток впервые продлило жиз...,Полина Лосева,2023-01-17,5.8,"Биология, Медицина",До сих пор этот метод проверяли только на моде...
3,Дельфины не смогли перекричать антропогенный шум,Сергей Коленов,2023-01-17,2.1,"Зоология, Экология и климат","Это очередное подтверждение того, насколько им..."
4,Счастливую юность связали со здоровым сердцем ...,Слава Гоменюк,2023-01-17,2.4,Медицина,Ученые собрали данные за 20 лет Взрослые амери...


Добавим содержательные названия столбцов:

In [ ]:
df.columns = ["title", "author", "date", "diffc", "rubrics", "text"]

Изменим тип столбца `diffc` на `float`, чтобы со значениями можно было проводить операции как с числами (например, построить гистограмму или отфильтровать/отсортировать):

In [ ]:
df["diffc"] = df["diffc"].astype(float)

In [ ]:
# описательные статистики как для числового столбца
df["diffc"].describe()

count    67.000000
mean      4.725373
std       2.283494
min       1.300000
25%       2.700000
50%       4.100000
75%       6.800000
max       8.900000
Name: diffc, dtype: float64

Выгрузим датафрейм в файл:

In [ ]:
df.to_excel("nplus1.xlsx")